In [2]:
from abc import ABCMeta, abstractmethod

class Strategy(object):

    __metaclass__ = ABCMeta

    @abstractmethod
    def generate_signals(self):
        raise NotImplementedError("Should implement generate_signals()!")

In [3]:
class QDLForecastingStrategy(Strategy):
    
    """ Requires:
    symbol - A stock symbol on which to form a strategy on
    bars - A DataFrame of bars for the above symbol
    start_period - beginning of the analysed time period
    start_test - beginning of the test window
    end_period - end of the analysed time period"""
    
    def __init__(self, symbol, bars, start_period, start_test, end_period, lags=5, lags_hilo=5):
        self.symbol = symbol
        self.bars = bars
        self.start_period = start_period
        self.start_test = start_test
        self.end_period = end_period   
        self.lags = list(range(1, lags+1))
        self.lags_hilo = list(range(1, lags_hilo+1))
        self.fit_model()

    def fit_model(self):
        """Fits a Quadratic Discriminant Analyser"""
        # Create a lagged series of the series
        snpret = create_lagged_series(self.symbol, self.bars, self.start_period, self.end_period, self.lags, self.lags_hilo) 

        # Use the prior two days of returns as 
        # predictor values, with direction as the response
        laglist = ["Lag%s" % str(i) for i in self.lags]
        hilo_laglist = ["HiLo Lag%s" % str(i) for i in self.lags_hilo]
        avg_vol = ["avg_vol_lags"]
        full_laglist = laglist + hilo_laglist + avg_vol
        X = snpret[full_laglist]
        y = snpret["Direction"]
        
        # Create training and test sets
        X_train = X[X.index < str(self.start_test)]
        y_train = y[y.index < str(self.start_test)]
        
        # Create the predicting factors for use 
        # in direction forecasting
        self.predictors = X[X.index >= str(self.start_test)]
        
        # Create the Quadratic Discriminant Analysis model
        # and the forecasting strategy
        self.model = QuadraticDiscriminantAnalysis()
        self.model.fit(X_train, y_train)
        score = self.model.score(X_train, y_train, sample_weight=None)
        return score
    
    def generate_signals(self):
        """Returns the DataFrame of symbols containing the signals
        to go long or short (1, -1)."""
        signals = pd.DataFrame(index=self.predictors.index)     

        # Predict the subsequent period with the QDA model
        signals['signal'] = self.model.predict(self.predictors)

        # Remove the first five signal entries to eliminate
        # NaN issues with the signals DataFrame
        signals['signal'][0:self.lags[-1]] = 0.0
        signals['positions'] = signals['signal'].diff()

        return signals

In [4]:
class RandomForecastingStrategy(Strategy):

    """Requires the symbol ticker and the pandas DataFrame of bars"""
    
    def __init__(self, symbol, bars):
        self.symbol = symbol
        self.bars = bars  
    
    def generate_signals(self):
        """Creates a pandas DataFrame of random signals."""
        signals = pd.DataFrame(index=self.bars.index)
        signals['signal'] = np.sign(np.random.randn(len(signals)))

        # The first five elements are set to zero in order to minimise
        # upstream NaN errors in the forecaster.
        signals['signal'][0:5] = 0.0
        return signals

In [5]:
class LongEveryDay(Strategy):
    
    def __init__(self, symbol, bars):
        """Requires the symbol ticker and the pandas DataFrame of bars"""
        self.symbol = symbol
        self.bars = bars

    def generate_signals(self):
        """Creates a pandas DataFrame of random signals."""
        signals = pd.DataFrame(index=self.bars.index)
        signals['signal'] = 1
        signals['signal'][0:5] = 0.0
        return signals

In [6]:
class MovingAverageCrossStrategy(Strategy):
    """    
    Requires:
    symbol - A stock symbol on which to form a strategy on.
    bars - A DataFrame of bars for the above symbol.
    short_window - Lookback period for short moving average.
    long_window - Lookback period for long moving average."""

    def __init__(self, symbol, bars, short_window=3, long_window=30):
        self.symbol = symbol
        self.bars = bars
        self.short_window = short_window
        self.long_window = long_window

    def generate_signals(self):
        """Returns the DataFrame of symbols containing the signals
        to go long, short or hold (1, -1 or 0)."""
        signals = pd.DataFrame(index=self.bars.index)
        signals['signal'] = 0.0

        # Create the set of short and long simple moving averages over the 
        # respective periods
        signals['short_mavg'] = bars['Close'].rolling(self.short_window, min_periods=1).mean()
        signals['long_mavg'] = bars['Close'].rolling(self.long_window,min_periods=1).mean()
        
        # Create a 'signal' (invested or not invested) when the short moving average crosses the long
        # moving average, but only for the period greater than the shortest moving average window
        signals['signal'][self.short_window:] = np.where(signals['short_mavg'][self.short_window:]
                                                         > signals['long_mavg'][self.short_window:], 1.0, -1.0)   

        # Take the difference of the signals in order to generate actual trading orders
        signals['positions'] = signals['signal'].diff()   

        return signals